In [1]:
import torch
import torch.nn as nn
import numpy as np

from basic_blocks import SetBlock,BasicConv2d,Non_local,Feed_Forward



In [3]:
_set_in_channels = 1
_set_channels = [32, 64, 128]
set_layer1 = SetBlock(BasicConv2d(_set_in_channels, _set_channels[0], 5, padding=2))
set_layer2 = SetBlock(BasicConv2d(_set_channels[0], _set_channels[0], 3, padding=1), True)

non_layer1 = SetBlock(Non_local( _set_channels[0],8))
non_layer2 = SetBlock(Non_local( _set_channels[0],4))


In [15]:
x=torch.randn(2,30,1,64,44)
pos_embedding_32=nn.Parameter(torch.randn(1,1,32,32,22))
bin_num=4

In [16]:
x=set_layer1(x)
print(x.shape)
x=set_layer2(x)
x_ori=x.clone()
x=x+pos_embedding_32
n,t,c,h,w=x.shape


win_size=h//bin_num
x=x.view(n,t,c,bin_num,win_size,w).permute(0,1,3,2,4,5)\
    .contiguous().view(n,t*bin_num,c,win_size,w) # n tp c h/p w   



print(x.shape)
x=non_layer1(x)
print(x.shape)
x=non_layer2(x)
print(x.shape)

x=x.view(n,t,bin_num,c,win_size,w).permute(0,1,3,2,4,5).contiguous().view(n,t,c,h,w)



torch.Size([2, 30, 32, 64, 44])
torch.Size([2, 120, 32, 8, 22])
torch.Size([2, 120, 32, 8, 22])
torch.Size([2, 120, 32, 8, 22])


In [17]:
x.shape

torch.Size([2, 30, 32, 32, 22])

In [2]:
_set_channels=[32,64,128]

In [3]:
net=SetBlock(Feed_Forward(_set_channels[1],_set_channels[2],_set_channels[2]))

In [4]:
x=torch.randn(2,30,64,8,22)

In [5]:
y=net(x)

In [6]:
y.shape

torch.Size([2, 30, 128, 8, 22])

In [2]:
import torch
import torch.nn as nn
x=nn.Parameter(torch.randn(3,2,1))

In [3]:
x[0]

tensor([[ 0.2198],
        [-0.2780]], grad_fn=<SelectBackward>)

In [4]:
x[1]

tensor([[-0.8669],
        [ 0.1025]], grad_fn=<SelectBackward>)

In [7]:
x[0].expand(-1,5)

tensor([[ 0.2198,  0.2198,  0.2198,  0.2198,  0.2198],
        [-0.2780, -0.2780, -0.2780, -0.2780, -0.2780]],
       grad_fn=<ExpandBackward>)

In [8]:
angle_probe=torch.tensor([[0.5,0.6],[0.3,0.2]])

In [9]:
angle_probe.shape

torch.Size([2, 2])

In [10]:
_,angle= torch.max(angle_probe, 1)

In [11]:
angle

tensor([1, 0])

In [15]:
x[angle].expand(-1,-1,6)

tensor([[[-0.8669, -0.8669, -0.8669, -0.8669, -0.8669, -0.8669],
         [ 0.1025,  0.1025,  0.1025,  0.1025,  0.1025,  0.1025]],

        [[ 0.2198,  0.2198,  0.2198,  0.2198,  0.2198,  0.2198],
         [-0.2780, -0.2780, -0.2780, -0.2780, -0.2780, -0.2780]]],
       grad_fn=<ExpandBackward>)